# HW3 - Model Based RL - Neural Dynamics Modeling, CEM, PETS



#### Background

**Model free vs model based**

HW1 and HW2 explored **model free RL**. In that setting, we don't learn a dynamics model or plan using a dynamics model - hence model free. Theoretically, model free MDP exploration takes place directly in the real world - in practice this only works for **perfect simulators** where the training environment exactly matches the testing environment, like video games or board games. These approaches **learn policies** that map all states to optimal actions.

This assignment explores **model based RL**. In this setting we **learn models** of the approximate transition dynamics from data and **plan actions with those models** - hence model based. This is required for robotics, autonomous driving, or any hardware system that is impossible to simulate perfectly.

Note, there is also a broad gray area between model based and model free rl. You can approximate dynamics and then learn a policy, you can plan trajectories in a perfect simulator, etc. This distinction between the algorithm classes is more historical than deeply significant.

**HW3**

This assignment builds to an important recent work in model based RL - [PETS](https://arxiv.org/abs/1805.12114): probabilistic ensembles with trajectory sampling (2018). It progresses from predecessor techniques: deterministic neural dynamics modeling (\~1990), Cross Entropy Method (\~1999), and stochastic neural dynamics modeling (\~1994).

For this assignment, we pretend our [inverted pendulum](https://gymnasium.farama.org/environments/mujoco/inverted_pendulum/) environment is an expensive hardware system we have previously collected data from. From that data we will train models, and with those models we will plan actions.

Note, variations of these algorithms exist. Please use the math contained in this notebook for the coding sections.

# 0. Warm Up Questions [30 pts total; 2 pt each]

1.    What're the differences between an optimal policy and an optimal trajectory?<br>

> An optimal policy $\pi$ knows how to create an optimal trajectory $\tau$, given different self/environment states. 

> `TODO`: check


2.    In model free RL we search over all possible policy parameters $\theta$ for optimal parameters $\theta^*$ which result in the highest sum of rewards:
$$
\theta^* = \arg \max_\theta \displaystyle\sum_{t=1}^{T} r(s_t, a_t)
$$
In basic language, what is the meaning of this constrained optimization below, which is used to plan trajectories in model based RL? [hint: the deterministic case](https://www.youtube.com/watch?v=4SL0DnxC1GM&list=PL_iWQOsE6TfVYGEGiAOMaOzzv41Jfm_Ps&index=40)
$$
a_1^*, ..., a^*_T = \arg \max_{a_1, ..., a_T} \displaystyle\sum_{t=1}^T r(s_t, a_t)\\
\text{  subject to:  } s_{t+1} = s_t + f_\theta(s_t, a_t)
$$ <br>

> The optimal trajectory ($a_1^*, ..., a^*_T$) is one that produces the maximum possible rewards ($r(s_t, a_t)$) given a situation whose state evolution (transition dynamics) is governed by the function $f_\theta(s_t, a_t)$.

> `TODO`: check


2.    Why is it helpful to replan at every time step in a Model Predictive Control settings? [hint: What if we make a mistake](https://www.youtube.com/watch?v=LkTmiylbHYk&list=PL_iWQOsE6TfVYGEGiAOMaOzzv41Jfm_Ps&index=45)<br>

> It allows the system to reduce the effects of error accumulation due to uncertainty in sensing and dynamics prediction — essentially turning it into a closed-loop feedback system. 

> `TODO`: check

3.    What're the 3 basic steps of model based RL? [hint: mbrl v 0.5](https://www.youtube.com/watch?v=LkTmiylbHYk&list=PL_iWQOsE6TfVYGEGiAOMaOzzv41Jfm_Ps&index=46)<br>

> 1. Obtain the dynamics model. 2. Interact with the environment. 3. Update the parameter weights.

> `TODO`: check

4.    Write a valid equation for mean squared error loss for a neural approximation to deterministic transition dynamics. Define any variables you use.<br>

> $\text{loss}_t = (s_{t+1} - \hat{s_{t+1}})^2$, where $\hat{s_{t+1}}$ is the predicted environment state at next step, and $s_{t+1}$ is the actual environment state.

5.    What're the two steps in the "guess & check" method of stochastic optimization? Define any variables you use. [hint](https://www.youtube.com/watch?v=pd9mKcH4kkk&list=PL_iWQOsE6TfVYGEGiAOMaOzzv41Jfm_Ps&index=41)<br>

6.    In general machine learning, what is an evolutionary algorithm?<br>

> An algorithm that utilizes the generation of a set of parameter candidates via random sampling and "natural selection" as the update rule. 

> `TODO`: check

7.    How does the cross entropy method improve on "guess & check"? What're the four steps of the cross entropy method? [hint](https://www.youtube.com/watch?v=pd9mKcH4kkk&list=PL_iWQOsE6TfVYGEGiAOMaOzzv41Jfm_Ps&index=41)<br>

> 

8.    In guess and check or CEM, you have an environment with 5 continuous states and 3 continuous actions and you want to randomly sample 1000 trajectories, each 15 timesteps long. What size must your mean and standard deviation tensors be to allow this sampling?<br>

> (5 + 3) * 15 * 1000 = shape (8, 15, 1000), size 120,000. 

> Wait, this is the data tensor size; since the mean/std are probably *reduced* over trajectories and timesteps, it's probably just 8?

> `TODO`: what about standard deviation?

9.    Alea is latin for dice. What is aleatoric uncertainty? <br>

> Uncertainty due to the inherent combinatorial complexity of the environmental state. 

> `TODO`: check

10.    What type of loss should you use to fit the parameters of a guassian to a set of random samples from a guassian? and why not just use MSE? [hint](https://pytorch.org/docs/stable/generated/torch.nn.GaussianNLLLoss.html)<br>

> 

11.    Episteme is greek for knowledge. What is epistemic uncertainty? <br>

> 

12.    In general machine learning, what is an ensemble?<br>

> A class of techniques that involve generation of a set of (intermediary) output value candidates and holistically combine them to obtain a better accuracy (vs. without ensembling). 

> `TODO`: check

13.    Why is sequential dynamics modeling harder than non sequential supervised learning?<br>

> Because there is an added variable of time and state history/memory. 

14. Explain the problem of distributional shift in model based RL, and write the equation that summarizes this phenomena. [hint: Does it work? No!](https://www.youtube.com/watch?v=LkTmiylbHYk&list=PL_iWQOsE6TfVYGEGiAOMaOzzv41Jfm_Ps&index=45)<br>

> 

15.    Why is uncertainty estimation important in model based RL? [hint: how uncertainty estimation can help](https://www.youtube.com/watch?v=pSvjDO1B9WY&list=PL_iWQOsE6TfVYGEGiAOMaOzzv41Jfm_Ps&index=47)<br>

> 



# Boiler plate

Read through at least once.

**Will hang until you upload the replay buffer** (every time you restart your runtime).

In [1]:
%%capture
# @title Imports

# !pip install gymnasium[mujoco]
# !apt install -y libgl1-mesa-glx libosmesa6 libglfw3 patchelf
import gymnasium as gym

import torch
from torch import nn, zeros
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
from collections import deque
import random
import copy

# code should work on either, faster on gpu
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# random seeds for reproducability
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
# @title Define Replay Buffer
class ReplayBuffer:
    def __init__(self):
        self.buffer = deque(maxlen=6000)
        self.batch_size = 32

    def store(self, state, action, reward, next_state, done):
        transitions = list(zip(state, action, reward, next_state, 1 - torch.Tensor(done)))
        self.buffer.extend(transitions)

    def sample(self):
        batch = random.sample(self.buffer, self.batch_size)
        # generic replay buffer from hw2, modified to only return (s, a, s')
        states, actions, rewards, next_states, not_dones = [torch.stack(e).to(device) for e in zip(*batch)]
        return states, actions, next_states

replay_buffer = ReplayBuffer()

In [3]:
# # @title Upload Replay Buffer
# from google.colab import files
# import pickle

# # Prompt the user to upload the file
# uploaded = files.upload()

# # Load the replay buffer from the uploaded file
# file_name = list(uploaded.keys())[0]  # Get the file name
# with open(file_name, 'rb') as f:
#     replay_buffer = pickle.load(f)

# print(f"Replay buffer loaded with {len(replay_buffer.buffer)} transitions!")


In [4]:
# @title Upload Replay Buffer
import pickle

# Load the replay buffer from the uploaded file
file_name = "./replay_buffer_hw3.pkl"
with open(file_name, 'rb') as f:
    replay_buffer = pickle.load(f)

print(f"Replay buffer loaded with {len(replay_buffer.buffer)} transitions!")

Replay buffer loaded with 6000 transitions!


In [5]:
# @title Visualization Code
import os
from gym.wrappers import RecordVideo
from IPython.display import Video, display, clear_output

# Force MuJoCo to use EGL for rendering (important for Colab)
# os.environ["MUJOCO_GL"] = "egl" # TEMPDEAC

def visualize(agent):
    """Visualize agent with a custom camera angle."""

    # Create environment in rgb_array mode
    env = gym.make("InvertedPendulum-v5", render_mode="rgb_array", reset_noise_scale=0.1, frame_skip=5)

    # Apply video recording wrapper
    env = RecordVideo(env, video_folder="./", episode_trigger=lambda x: True)

    obs, _ = env.reset()

    # Access the viewer object through mujoco_py
    viewer = env.unwrapped.mujoco_renderer.viewer  # Access viewer
    viewer.cam.distance = 3.0     # Set camera distance
    viewer.cam.azimuth = 90       # Rotate camera around pendulum
    viewer.cam.elevation = 0   # Tilt the camera up/down


    for t in range(200):
        with torch.no_grad():
            actions = agent.get_action(torch.Tensor(obs).to(device)[None, :])[:, 0]
        obs, _, done, _= env.step(actions.cpu().numpy())
        if done:
            break
    env.close()

    # Display the latest video
    clear_output(wait=True)
    display(Video("./rl-video-episode-0.mp4", embed=True))

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [6]:
# @title Evaluation Code

def evaluate(agent):

    # Create environment in rgb_array mode
    env = gym.make("InvertedPendulum-v5", reset_noise_scale=0.1, frame_skip=5)

    n = 3
    mean_duration = 0
    for i in range(n):
        obs, _ = env.reset()
        done, t = False, 0
        while not done and t < 200:
            with torch.no_grad():
                actions = agent.get_action(torch.Tensor(obs).to(device)[None, :])[:, 0]
            obs, _, done, _, _ = env.step(actions.cpu().numpy())
            t += 1

        mean_duration += t
        print(f"trial {i+1}/{n} lasted {t*.1:.3f} seconds")

    env.close()
    print(f"\nmean duration: {(mean_duration * .1 / n):.3f} seconds")


#Tensorboard

In [7]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir runs

# Deterministic Dynamics [20 pts]

#### Problem Statement


1.   Define the dynamics network $f_\theta$ and optimizer [5 pts]
2.   Define `predict()` and `get_loss()` [5 pts]
3.   Finish the training code and run training (code cell below unit test) [5 pts]
3.   Conceptual question [5 pts]
________________________________________
Deterministic Neural Dynamics Modeling

Let's start by training a neural network to represent the dynamics of the cartpole system from HW2 using 10 mintues of data collected at 10 Hz from a single robot (6000 transitions). We can assume the data came from a human attemping to control the cartpole with a joystick, but it's suboptimal in terms of performance.

We will model the transition function as:

$$
s_{t+1} = s_t + f_\theta(s_t, a_t)
$$
Or equivalently:
$$
s_{t+1} - s_t = f_\theta(s_t, a_t)
$$

Train $f_\theta(s_t, a_t)$ with Mean Squared Error loss on minibatches from the dataset of transition tuples $\mathcal{D} = \{(s, a, s')\}$

_______________________________________

Deterministic Neural Dynamics Modeling Loss:

$$
\mathcal{L}(\theta) = \mathbb{E}[\{(s_{t+1} - s_t)  - f_\theta(s_t, a_t)\}^2]
$$


Where:
- $ \mathcal{L} $ is the dynamics net loss; a function of network parameters $\theta$
-$s_{t+1}$ is state at timestep $t+1$
-$s_t$ is action at timestep $t$
-$f_\theta(s_t, a_t)$ is the dynamics network parametrized by $\theta$
-$\mathbb{E}$ is the expectation or average over the minibatch

(hint: No for loops. Use torch's batched operations for greater training speed.)

#### Code

In [8]:
n_hidden = 100

class DeterministicDynamics():
    # NOTE: experiment with learning rate and model size, but keep it same for all models
    def __init__(self, n_obs, n_actions):
        torch.manual_seed(0)
        
        self.model = nn.Sequential(
          nn.Linear(n_obs + n_actions, n_hidden),
          nn.ReLU(),
          nn.Linear(n_hidden, n_hidden),
          nn.ReLU(),
          nn.Linear(n_hidden, n_obs),
          # nn.ReLU(), # turned off, since transition dynamics is delta-encoded
        )
        # end student code

    def predict(self, states, actions):
        return self.model(torch.cat([states, actions], dim=-1))
        #end student code

    def get_loss(self, states, actions, next_states):
        delta_states = next_states - states
        loss = F.mse_loss(self.predict(states, actions), delta_states)
        return loss
        # end student code

In [9]:
# @title Unit Tests

def determ():
    torch.manual_seed(0)
    # these dont match an actual rollout..
    # print debug values during training loop rather than unit tests
    batch_size, n_obs, n_actions = 2, 4, 1
    s = torch.rand((batch_size, n_obs))
    a = torch.rand((batch_size, n_actions))
    s_ = torch.rand((batch_size, n_obs))

    dyn = DeterministicDynamics(4, 1)
    torch.manual_seed(0)
    dyn.model = nn.Linear(5, 4) # you should not use this architecture..

    delta = dyn.predict(s, a)
    # print(delta)
    expected_delta = torch.tensor([[ 0.1906,  0.5041, -0.3875,  0.3737],
        [-0.2708,  0.6156, -0.7808,  0.1884]])
    assert torch.allclose(delta, expected_delta, atol=1e-4), \
    "Deterministic dynamics predict does not match expected value."
    print("Test passed: Deterministic dynamics predict appears correct!")


    loss = dyn.get_loss(s, a, s_)
    # print(loss)
    assert abs(loss.item() - (0.3435)) < 1e-4, \
    "Deterministic dynamics loss does not match expected value."
    print("Test passed: Deterministic dynamics loss appears correct!")

determ()

Test passed: Deterministic dynamics predict appears correct!
Test passed: Deterministic dynamics loss appears correct!


In [10]:
# writer = SummaryWriter(log_dir=f'runs/deterministic')
# # NOTE: if you need extra hints here check out the analagous code in HW1 and HW2

# # params
# env = gym.make("InvertedPendulum-v5")
# lr = 1e-4

# n_obs = env.observation_space.shape[0]
# n_actions = env.action_space.shape[0]
# deterministic_dynamics = DeterministicDynamics(n_obs, n_actions)
# deterministic_dynamics.model.train()
# optimizer = torch.optim.AdamW(deterministic_dynamics.model.parameters(), lr=lr, weight_decay=1e-2)

# for i in range(30_000): # train over 30,000 minibatch samples
#     states, actions, next_states = replay_buffer.sample()
    
#     # pred = deterministic_dynamics.predict(states, actions)
#     optimizer.zero_grad()  # reset gradient
#     loss = deterministic_dynamics.get_loss(states, actions, next_states)
#     loss.backward()

#     torch.nn.utils.clip_grad_norm_(deterministic_dynamics.model.parameters(), 1.0)  # optional
#     optimizer.step()

#     # end student code
#     writer.add_scalar("stats/mse_loss", loss.item(), i)


#### Conceptual Question

1. In the context of RL for physical control, why learn a dynamics model rather than (1) training directly on a hardware system or (2) deriving a simulator for our system by hand?<br>

> (1) Direct training on hardware system is likely to be prohibitively expensive and dangerous (i.e., you *don't* want to roll out random actions into a physical robot). 

> (2) Not all physical phenomena can be easily implemented in a physics simulator; for example — fluid and soft-body mechanics are extremely hard to derive/integrate, even numerically. Model-based RL allows you to insert some of the knowledge you know (e.g., physical equations derived by great mathematicians and physicists) while retaining the data-driven aspect that is suited for handling real-world chaos and complexity. 


# Cross Entropy Method [20 pts]

#### Problem Statement


1.   Finish implementing CEM `get_action()` and run deterministic CEM [15 pts]
2.   Conceptual question [5 pts]
_______________________________

Background

Now that we have a model, it's time to plan. We will use the Cross Entropy Method - possibly the simplest online planning algorithm. It's a sampling based evolutionary algorithm that iteratively reduces the cross entropy between our distribution over actions and the optimal disribution over actions at each time step. We start with a random distribution over actions, sample from it, simulate the outcomes and calculate their returns, grab a subset of the best performing actions, and use those to refit our random distribution over actions. Repeat until convergence (in theory) or for a fixed number of iterations (in practice). Replan at every timestep. For further explanation check out the hints in the warm up questions.

We are doing open loop planning by solving this constrained optimization:

$$
a_1^*, ..., a_T^* = \arg\max_{a_1, ..., a_T} \sum_{t=1}^{T} r_t  \\
\quad \quad \text{subject to} \quad s_{t+1} = s_t + f_\theta(s_t, a_t)
$$

_______________________________
Pseudocode for CEM


A. Initialize distribution
- Create mean = 0, std_dev = 1 tensors for T timesteps and n_action dimensions  (done for you)

B. Loop over CEM iterations:  

- Randomly sample n_samples parallel action sequences (done for you)

- Loop over timesteps T
     - Simulate the action sequences in parallel using $f_\theta$   
     - Use `torch.no_grad()` to disable gradient tracking
     - Accumulate undiscounted returns at each timestep

- Select elite actions:  
     - Choose the n_elite action sequences with the highest returns

- Refit distribution:  
     - Refit mean and std_dev to the n_elite actions
     - Clamp the minimum std_dev to .5 to prevent collapse and ensure exploration

C. MPC return
- Return the mean action for the first timestep only
- It should be a tensor of size `torch.Size([1, n_actions])`, since our batching dimension is 1

(hint: understand dimensions of all tensors)




         



#### Code

In [11]:
from torch.distributions import Normal

class CEM:
    def __init__(self, dynamics):
        self.dynamics: DeterministicDynamics = dynamics
        self.T = 10  # planning time horizon
        self.n_actions = 1
        self.cem_iter = 20
        self.n_samples = 1000
        self.n_elite = 200

    def compute_rewards(self, state):
        reward_t = -state[..., 0].square() / 5.  # incremental position penalty
        reward_t -= state[..., 1].square()  # incremental angle penalty
        # angle and position limits
        soft_done = torch.logical_or(state[..., 1].abs() > 0.2, state[..., 0].abs() > 1.)
        reward_t +=  1 - 2 * soft_done.float()
        return reward_t

    def get_action(self, initial_states):
        mean = torch.zeros((self.T, self.n_actions))
        std_dev = torch.ones((self.T, self.n_actions))

        for i in range(self.cem_iter):
            # initial random rollout (before CEM)
            states = initial_states.repeat(self.n_samples, 1)
            actions = Normal(mean, std_dev).sample((self.n_samples,)).clamp(-3, 3).to(device)
            returns = torch.zeros((self.n_samples,))

            # roll out actions in batch (for T steps)
            for j in range(self.T):  # using for loop since chronological dependency and low loop repeat n
                with torch.no_grad():
                    next_states_pred = self.dynamics.predict(states, actions[:,j,...])  # sa: state & action
                states = next_states_pred  # alias (for loop update)
                returns += self.compute_rewards(states)

            # choose elites & re-fit
            returns_sorted, idx_sorted = returns.sort(dim=0, descending=True)
            elite_actions = actions[idx_sorted[:self.n_elite], ...]
            elite_std, elite_mean = torch.std_mean(elite_actions, dim=0)
            mean, std_dev = elite_mean, elite_std  # alias (for loop update)
            std_dev = std_dev.clamp_min(0.5)  # min_clamp std to ensure exploration

        # return actions[idx_sorted[0], 0, ...].unsqueeze(-1)
        return mean[0].unsqueeze(-1)
        # end student code


In [12]:
# @title Unit Tests

def cem():
    torch.manual_seed(0)
    # these dont match an actual rollout..
    # print debug values during training loop rather than unit tests
    batch_size, n_obs = 1, 4
    s = 2*torch.rand((batch_size, n_obs)).to(device)-3

    dyn = DeterministicDynamics(4, 1)
    torch.manual_seed(0)
    dyn.model = nn.Linear(5, 4).to(device) # you should not use this architecture..
    cem = CEM(dyn)
    a = cem.get_action(s)

    assert a.shape == torch.Size([1, 1]), \
    "Action size appears wrong"

    # print(a.item())
    if device == 'cpu':
        assert abs(a.item() -  (-1.4559649229049683)) < 1e-4, \
        "Action value appears wrong"
    else: #todo fix this...
        assert abs(a.item() -  (-1.4642404317855835)) < 1e-4, \
        "Action value appears wrong"

    print("Test passed: CEM Action appears correct!")

cem()

AssertionError: Action value appears wrong

In [73]:
cem = CEM(deterministic_dynamics)
visualize(cem)
evaluate(cem)

NameError: name 'deterministic_dynamics' is not defined

#### Conceptual Questions
1. What're the pros and cons of real time planning (like this CEM method) vs policy learning (like REINFORCE, DQN, PPO, etc)?<br>
\
Type answer here...

# Stochastic Dynamics [15 pts]

#### Problem Statement

1.   Define the stochastic dynamics network $f_\theta$ and optimizer [5 pts]
2.   Define `predict()` and `get_loss()` [5 pts]
3.   Finish the training code, run training, run stochastic CEM [5 pts]
3.   Conceptual question [5 pts]
___________________________________
Background

We know trajectory optimization over learned models introduces challeneges due to distribution shift and error accumulation. We also know this problem is worse when modeling with neural nets. Let's try modeling our dynamics as a stochastic system and planning in expectation over the dynamics. We will theoretically model the state transition function as:

$$
\mu_t, \sigma_t = f_\theta(s_t, a_t)\\
s_{t+1} - s_t \sim \mathcal{N}(\mu_t, \sigma_t)
$$


Where mean $\mu_t$ and standard deviation $\sigma_t$ form normal distribution $\mathcal{N}$ from which transitions are sampled. In practice we have to use these slightly less elegant equations to keep $\sigma_t$ positive and bounded:

$$
\mu_t, \log \sigma_t = f_\theta(s_t, a_t)\\
\sigma_t = \text{clamp} (e^{\log \sigma_t}, 10^{-8}, 10)  \\
s_{t+1} - s_t \sim \mathcal{N}(\mu_t, \sigma_t)
$$

Where:  
- $ \mu_t $ is the mean of the predicted state difference
- $ \log \sigma_t $ is the log of the standard deviation, modeling the uncertainty in the transition
- $ e^{\log \sigma_t}$ ensures positivity
- $\text{clamp}()$ ensures numerical stability
-  $\sim$ denotes random sampling
- $\mathcal{N}$ denotes Normal aka Guassian Distribution

___________________________________

Stochastic Neural Dynamics Loss  

Instead of minimizing a simple squared error loss, we now minimize the **Gaussian Negative Log-Likelihood (NLL) loss**, which accounts for both the predicted mean and uncertainty:  

$$
\mathcal{L}(\theta) = \mathbb{E} \left[ \frac{((s_{t+1} - s_t) - \mu_t)^2}{2\sigma_t^2} + \log \sigma_t \right]
$$

where:  
- $ \mathcal{L}(\theta) $ is the stochastic dynamics loss, a function of network parameters $ \theta $
- $ s_{t+1} $ is the state at timestep $ t+1 $
- $ s_t $ is the state at timestep $ t $
- $ \mu_t $ is the mean prediction of the state transition
- $ \sigma_t $ is the predicted standard deviation
- $ \mathbb{E} $ denotes the expectation (average over the minibatch)

(hint: don't actually implement that equation)

#### Code

In [ ]:
from torch.distributions import Normal

class StochasticDynamics():
    def __init__(self, n_obs, n_actions):
        torch.manual_seed(0)
        # Student code here
        self.model = None

        # end student code

    def predict(self, states, actions):
        # Student code here


        return None
        # end student code

    def get_loss(self, states, actions, next_states):
        # Student code here


        return None
        # end student code


In [ ]:
# @title Unit Tests

def stoch():
    torch.manual_seed(0)
    # these dont match an actual rollout..
    # print debug values during training loop rather than unit tests
    batch_size, n_obs, n_actions = 2, 4, 1
    s = torch.rand((batch_size, n_obs))
    a = torch.rand((batch_size, n_actions))
    s_ = torch.rand((batch_size, n_obs))

    dyn = StochasticDynamics(4, 1)
    torch.manual_seed(0)
    dyn.model = nn.Linear(5, 8) # you should not use this architecture..

    delta = dyn.predict(s, a)
    # print(delta)
    expected_delta = torch.tensor([[ 0.2525,  1.1405, -1.9571, -1.1799],
        [-0.4345, -0.4267, -0.1200, -2.6686]])
    assert torch.allclose(delta, expected_delta, atol=1e-4), \
    "Stochastic dynamics predict does not match expected value."
    print("Test passed: Stochastic dynamics predict appears correct!")


    loss = dyn.get_loss(s, a, s_)
    # print(loss)
    assert abs(loss.item() - (0.1352)) < 1e-4, \
    "Stochastic dynamics loss does not match expected value."
    print("Test passed: Stochastic dynamics loss appears correct!")

stoch()

In [ ]:
writer = SummaryWriter(log_dir=f'runs/stochastic')
# student code here
stochastic_dynamics = None


for i in range(30_000):


    # end student code
    writer.add_scalar("stats/nll_loss", loss.item(), i)


You don't have to change any CEM code, but if you implemented the sampling based `predict()` function, then you're now solving the planning problem in expectation over stochastic state transitions:

$$
a_1^*, ..., a_T^* = \arg\max_{a_1, ..., a_T} \mathbb{E}_{s_{t+1} \sim p(s_{t+1} | s_t, a_t)} \left[ \sum_{t=1}^{T} r_t \right]
 \\
\quad \quad \text{subject to} \quad s_{t+1} - s_t \sim \mathcal{N}(\mu_t, \sigma_t)
$$

In [ ]:
cem = CEM(stochastic_dynamics)
visualize(cem)
evaluate(cem)

#### Conceptual Questions
1. What type of uncertanity does our stochastic dynamics model capture? Is this the right type of uncertainty to use for modeling our system? Why or why not.<br>
\
Type answer here...

2. Did stochastic modeling help or hurt your mean duration relative to deterministic modeling on this problem? Hypothesize why. (Either outcome is fine - your answer just has to make sense.)<br>
\
Type answer here...


# Ensembled Dynamics [15 pts]

#### Problem Statement

1.   Define a list of dynamics networks $f_{\theta_j}$, a single optimizer, batch `predict()`, and `get_loss(..., j)`, finish and run training code [5 pts]
2.   Update CEM for ensembled dynamics and run it [5 pts]
3.   Conceptual questions [5 pts]
________________________________________
Background

Modelling our system as stochastic doesn't capture all types of uncertainty. Let's try a different technique. Train 5 deterministic dynamics models, and plan in expectation over all of them. We will model the transition functions as:

$$
s_{t+1} - s_t = f_{\theta_j}(s_t, a_t), \quad \forall j \in \{1, 2, 3, 4, 5\}
$$

Where each $f_{\theta_j}$ is independently initialized and trained on independent minibatches from $\mathcal{D} = \{(s, a, s')\}$.

___________________________________

Ensembled Dynamics Modeling Loss:

$$
\mathcal{L}(\theta) = \sum_{j=1}^{N} \mathbb{E}[\{(s_{t+1} - s_t)  - f_{\theta_j}(s_t, a_t)\}^2]
$$

Where $ (s, a, s') $ are sampled independently from $ \mathcal{D}$ for each network $f_{\theta_j}$.


And:
- $ \mathcal{L} $ is the total ensemble loss; a function of all $j$ network parameters $\theta_j$
- $N$ is the number of networks in the ensemble
-$s_{t+1}$ is state at timestep $t+1$
-$s_t$ is action at timestep $t$
-$f_{\theta_j}(s_t, a_t)$ is the $j^{th}$ dynamics network parametrized by $\theta_j$
-$\mathbb{E}$ is the expectation or average over the minibatch


#### Code



In [ ]:
class EnsembleDynamics():
    def __init__(self, n_obs, n_actions):
        torch.manual_seed(0)
        self.N = 5
        # Student code here
        self.models = None

        # end student code

    # first dimension of states and return should be N
    def predict(self, states, actions):
        # student code here
        return None
        # end student code


    def get_loss(self, states, actions, next_states, j):
        # student code here
        return None # loss of jth model
        # end student code


In [ ]:
# @title Unit Tests (Non Exhaustive)

def ensemb():
    torch.manual_seed(0)
    # these dont match an actual rollout..
    # print debug values during training loop rather than unit tests
    batch_size, n_obs, n_actions, N = 2, 4, 1, 5
    s = torch.rand((N, batch_size, n_obs))
    a = torch.rand((batch_size, n_actions))
    s_ = torch.rand((N, batch_size, n_obs))

    dyn = EnsembleDynamics(4,1)
    torch.manual_seed(0)
    dyn.models = [nn.Linear(5, 4)]*5 # you should not do this ..

    delta = dyn.predict(s, a)
    # print(delta)
    expected_delta = torch.tensor([[[ 0.1784,  0.5125, -0.4006,  0.3599],
         [-0.2039,  0.5696, -0.7091,  0.2641]],

        [[-0.0387,  0.5905, -0.5743,  0.3233],
         [-0.1179,  0.4850, -0.4340,  0.2391]],

        [[ 0.1092,  0.5561, -0.5061,  0.3428],
         [-0.0961,  0.5799, -0.7381,  0.3018]],

        [[-0.1596,  0.8154, -0.7121,  0.5816],
         [-0.2096,  0.4903, -0.6012,  0.1556]],

        [[ 0.0700,  0.5045, -0.3103,  0.3136],
         [-0.1676,  0.6439, -0.7756,  0.4240]]])
    assert torch.allclose(delta, expected_delta, atol=1e-4), \
    "Ensemble dynamics predict does not match expected value."
    print("Test passed: Ensemble dynamics predict appears correct!")


    loss = dyn.get_loss(s[0], a, s_[0], 0)
    # print(loss)
    assert abs(loss.item() - (0.4017)) < 1e-4, \
    "Ensemble dynamics loss does not match expected value."
    print("Test passed: Ensemble dynamics loss appears correct!")

ensemb()

In [ ]:
writer = SummaryWriter(log_dir=f'runs/ensembled')

# student code here
ensemble_dynamics = None

for i in range(30_000):

    # end student code
    writer.add_scalar("stats/mse_loss", loss.item() / ensemble_dynamics.N, i)

Update your CEM implementation below so it solves the following constrained optimization in expectation over the $N$ models in your ensemble:


\begin{aligned}
a_1^*, ..., a_T^* &= \arg\max_{a_1, ..., a_T} \frac{1}{N} \sum_{j=1}^{N} \sum_{t=1}^{T} r_t^{(j)} \\
\\
\text{subject to} \quad s_{t+1}^{(j)} &= s_t^{(j)} + f_{\theta_j}(s_t^{(j)}, a_t), \quad \forall j \in \{1, ..., N\} \\
\\
\text{given} \quad s_0^{(j)} &= s_0, \quad \forall j \in \{1, ..., N\}
\end{aligned}


In [ ]:
# copy paste your CEM code from above, and update it for ensembles
# student code here


# end student code

In [ ]:
# @title Unit Tests (Non Exhaustive)

def ensemble_cem():
    torch.manual_seed(0)
    # these dont match an actual rollout..
    # print debug values during training loop rather than unit tests
    batch_size, n_obs = 1, 4
    s = 2*torch.rand((batch_size, n_obs)).to(device)-3

    dyn = EnsembleDynamics(4, 1)
    torch.manual_seed(0)
    dyn.models = [nn.Linear(5, 4).to(device)]*5 # you should not do this ..
    cem = CEM(dyn)
    a = cem.get_action(s)

    assert a.shape == torch.Size([1, 1]), \
    "Action size appears wrong"

    # print(a.item())
    if device == 'cpu':
        assert abs(a.item() -  (-1.4559649229049683)) < 1e-4, \
        "Action value appears wrong"
    else: #todo fix this...
        assert abs(a.item() -  (-1.4642404317855835)) < 1e-4, \
        "Action value appears wrong"
    print("Test passed: CEM Action appears correct!")

ensemble_cem()

In [ ]:
cem = CEM(ensemble_dynamics)
visualize(cem)
evaluate(cem)

#### Conceptual Question

1. What type of uncertanity does our ensembled dynamics model capture? Is this the right type of uncertainty for our problem? Why or why not.<br>
\
Type answer here...

2. Did ensembled modeling help or hurt your mean duration relative to deterministic modeling on this problem? Hypothesize why. (Either outcome is fine - your answer just has to make sense.)<br>
\
Type answer here...